In [1]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.contrib.layers import flatten, conv2d, fully_connected
from collections import deque, Counter
import random
from datetime import datetime
from IPython import display
import matplotlib.pyplot as plt
import time
from hide_and_seek_2thief import HideAndSeekEnv
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 환경 설정
env = HideAndSeekEnv()
n_outputs = env.action_space.n
env.render()

o - - - - - - - - - - - - - - - - - o
| P |       |           |           |
|           | - |   |   | - | - |   |
|   |         3     |   |           |
| - |                       | - |   |
|       |   |                       |
|   |                       | - |   |
|   |                 0     |       |
|   | - | - | - |           | - |   |
|                               | - |
o - - - - - - - - - - - - - - - - - o



In [3]:
# Q 네트워크를 구축하기 위해 q_network라는 함수를 정의
# Q 네트워크를 입력하고 해당 상태의 모든 작업에 대한 Q 값을 얻는다.
# fully connected layer가 이어지는 동일한 패딩을 가진 3개의 convolutional layers로 Q 네트워크를 구축
tf.reset_default_graph()

def q_network(X, name_scope):
    #layers 초기화
    initializer = tf.contrib.layers.variance_scaling_initializer()
    
    with tf.variable_scope(name_scope) as scope:
        #convolutional layers 초기화
        layer_1 = conv2d(X, num_outputs=32, kernel_size=(5,5), stride=4, padding="SAME", weights_initializer=initializer)
        tf.summary.histogram('layer_1',layer_1)
        
        layer_2 = conv2d(layer_1, num_outputs=64, kernel_size=(3,3), stride=2, padding="SAME", weights_initializer=initializer)
        tf.summary.histogram('layer_2',layer_2)
        
        layer_3 = conv2d(layer_2, num_outputs=64, kernel_size=(3,3), stride=1, padding="SAME", weights_initializer=initializer)
        tf.summary.histogram('layer_3',layer_3)
        
        # fully connected layer에 공급하기 전에 layer_3의 결과를 평탄화
        flat = flatten(layer_3)
        
        fc = fully_connected(flat, num_outputs=128, weights_initializer=initializer)
        tf.summary.histogram('fc',fc)
        
        output = fully_connected(fc, num_outputs=n_outputs, activation_fn=None, weights_initializer=initializer)
        tf.summary.histogram('output',output)
        
        #vars는 가중치와 같은 네트워크 매개변수를 저장
        vars = {v.name[len(scope.name):]: v for v in tf.get_collection(key=tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope.name)}
        return vars, output

In [4]:
# 엡실론 그리디 정책을 수행하기 위해 epsilon_greedy라는 함수를 정의
# 영원히 탐색하고 싶지 않기 때문에 엡실론의 가치가 시간이 지남에 따라 쇠퇴하는 쇠퇴 엡실론 탐욕 정책을 사용
# 즉, 시간이 지남에 따라 우리 정책은 좋은 행동만 이용할 것입니다.
eps_min=0.05
eps_max=0.5
eps_decay_steps = 500000

def epsilon_greedy(action, step):
    p = np.random.random(1).squeeze()
    epsilon = max(eps_min, eps_max - (eps_max-eps_min) * step/eps_decay_steps)
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)
    else:
        return action

In [5]:
# 경험을 보유하는 50000의 경험 버퍼를 초기화
# 에이전트의 모든 경험, 즉 (상태, 행동, 보상)을 경험 버퍼에 저장하고 네트워크 훈련을 위해 이 경험의 미니배치에서 샘플링
buffer_len = 500000
exp_buffer = deque(maxlen=buffer_len)

# 메모리에서 경험을 샘플링하기 위해 sampled_memories라는 함수를 정의
# 배치 크기는 메모리에서 샘플링된 경험의 수이다.
def sample_memories(batch_size):
    perm_batch = np.random.permutation(len(exp_buffer))[:batch_size]
    mem = np.array(exp_buffer)[perm_batch]
    return mem[:,0],mem[:,1],mem[:,2],mem[:,3],mem[:,4]

In [6]:
# 하이퍼파라미터 정의
num_episodes = 1000
batch_size = 48
input_shape = (None, 11, 19, 1)
learning_rate = 0.001
X_shape = (None, 11, 19, 1)
discount_factor = 0.97

global_step = 0
copy_steps = 100
steps_train = 4
start_steps = 2000

In [7]:
logdir = 'ch8_logs'
tf.reset_default_graph()

# 입력에 대한 게임 상태를 정의
X = tf.placeholder(tf.float32, shape=X_shape)

# 교육을 토글하기 위해 in_training_model이라는 부울을 정의
in_training_mode = tf.placeholder(tf.bool)

In [8]:
# 기본 대상 Q 네트워크를 구축
# 입력 X를 취하고 상태의 모든 작업에 대해 Q 값을 생성하는 Q 네트워크를 구축
mainQ, mainQ_outputs = q_network(X, 'mainQ')

# 목표 Q 네트워크를 구축
targetQ, targetQ_outputs = q_network(X, 'targetQ')

In [9]:
# 행동 값에 대한 자리 표시자 정의
X_action = tf.placeholder(tf.int32, shape=(None,))
Q_action = tf.reduce_sum(targetQ_outputs * tf.one_hot(X_action, n_outputs), axis=-1,keep_dims=True)

In [10]:
# 기본 Q 네트워크 매개변수를 대상 Q 네트워크에 복사
copy_op = [tf.assign(main_name, targetQ[var_name]) for var_name, main_name in mainQ.items()]
copy_target_to_main = tf.group(*copy_op)

In [11]:
# gradient descent optimizer를 사용하여 손실 계산 및 최적화

# 행동에 대한 자리 표시자를 정의
y = tf.placeholder(tf.float32, shape=(None,1))

# 실제 값과 예측 값의 차이인 손실을 계산
loss = tf.reduce_mean(tf.square(y - Q_action))

# loss을 최소화하기 위해 adam optimizer를 사용
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

loss_summary = tf.summary.scalar('Loss', loss)
merge_summary = tf.summary.merge_all()
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [12]:
# tensorflow와 그 안의 모델을 실행
with tf.Session() as sess:
    init.run()
    
    # 로그를 기록할 파일 열기
    log_file = "log.txt"
    
    #에피소드
    for i in range(num_episodes):
        done = False
        obs = env.reset()
        epoch = 0
        episodic_reward = 0
        actions_counter = Counter()
        episodic_loss = []
        
        #상태가 최종 상태가 아닌 동안
        while not done:
            # 전처리된 게임 화면 가져오기
            obs = env.render('dqn')
            
            # 게임 화면을 피드하고 각 작업에 대한 Q 값을 가져오기
            actions = mainQ_outputs.eval(feed_dict={X:[obs], in_training_mode: False})
            
            # 행동 가져오기
            action = np.argmax(actions, axis =-1)
            actions_counter[str(action)] += 1
            
            #엡실론 그리디 정책을 사용하여 행동 선택
            action = epsilon_greedy(action, global_step)
            
            #행동을 수행하고 다음 상태인 next_obs로 이동하여 보상을 받는다.
            next_obs, reward, done, _ = env.step(action)
            
            #이 전환을 재생 버퍼에 경험으로 저장
            exp_buffer.append([obs, action, env.render('dqn'), reward, done])
            
            #특정 단계 후에 경험 버퍼의 샘플로 Q 네트워크를 훈련
            if global_step % steps_train == 0 and global_step > start_steps:
                # 샘플 경험
                o_obs, o_act, o_next_obs, o_rew, o_done = sample_memories(batch_size)
                
                # 상태
                o_obs = [x for x in o_obs]
                
                # 다음 상태
                o_next_obs = [x for x in o_next_obs]
                
                # 다음 행동
                next_act = mainQ_outputs.eval(feed_dict={X:o_next_obs, in_training_mode:False})
                
                # 보상
                y_batch = o_rew + discount_factor * np.max(next_act, axis=-1) * (1-o_done)
                
                # 모든 요약을 병합하고 파일에 쓰기
                mrg_summary = merge_summary.eval(feed_dict={X: o_obs, y:np.expand_dims(y_batch, axis=-1), X_action:o_act, in_training_mode:False})
                file_writer.add_summary(mrg_summary, global_step)
                
                # 네트워크 훈련 및 loss 계산
                train_loss, _ = sess.run([loss, training_op], feed_dict={X:o_obs, y:np.expand_dims(y_batch, axis=-1), X_action:o_act, in_training_mode: True})
                episodic_loss.append(train_loss)
                
            # 일정 간격 후에 주요 Q 네트워크 가중치를 대상 Q 네트워크에 복사
            if (global_step+1)%copy_steps == 0 and global_step > start_steps:
                copy_target_to_main.run()
                    
            epoch += 1
            global_step += 1
            episodic_reward += reward
            
        # 파일에 로그 기록
        with open(log_file, "a") as file:
            file.write("Epoch: " + str(epoch) + ", Reward: " + str(episodic_reward))
            file.write(", Loss: " + str(np.mean(episodic_loss)))
            file.write(", Actions: " + str(actions_counter) + "\n")
        
        # 정보 출력
        print('Epoch', epoch, 'Reward', episodic_reward)
        
    # 저장 객체 생성
    saver = tf.train.Saver()
    
    # 모델 저장
    saver.save(sess, './model', global_step=global_step)

/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 971 Reward -6462
Epoch 829 Reward -5537
Epoch 709 Reward -4472
Epoch 606 Reward -3685
Epoch 235 Reward -731
Epoch 842 Reward -5676
Epoch 341 Reward -1822
Epoch 504 Reward -2539
Epoch 3022 Reward -20321
Epoch 739 Reward -2738
Epoch 190 Reward -479
Epoch 933 Reward -5182
Epoch 1302 Reward -6091
Epoch 1174 Reward -3668
Epoch 1151 Reward -4122
Epoch 261 Reward -460
Epoch 109 Reward 286
Epoch 413 Reward -954
Epoch 609 Reward -2810
Epoch 155 Reward -349
Epoch 580 Reward -1184
Epoch 605 Reward -2253
Epoch 369 Reward -631
Epoch 431 Reward -1026
Epoch 283 Reward -338
Epoch 418 Reward -1490
Epoch 386 Reward -1062
Epoch 284 Reward -501
Epoch 1748 Reward -6294
Epoch 408 Reward -1759
Epoch 404 Reward -981
Epoch 462 Reward -1624
Epoch 172 Reward -308
Epoch 64 Reward 376
Epoch 215 Reward -396
Epoch 386 Reward -1143
Epoch 446 Reward -1734
Epoch 338 Reward -1230
Epoch 451 Reward -1136
Epoch 455 Reward -1527
Epoch 1198 Reward -3728
Epoch 828 Reward -2890
Epoch 833 Reward -2706
Epoch 147 Reward -31

Epoch 74 Reward 213
Epoch 50 Reward 291
Epoch 40 Reward 463
Epoch 271 Reward -353
Epoch 106 Reward 343
Epoch 305 Reward -414
Epoch 179 Reward 9
Epoch 147 Reward 149
Epoch 184 Reward 139
Epoch 559 Reward -2054
Epoch 61 Reward 415
Epoch 43 Reward 433
Epoch 101 Reward 132
Epoch 292 Reward -626
Epoch 969 Reward -3382
Epoch 70 Reward 316
Epoch 55 Reward 403
Epoch 517 Reward -1904
Epoch 381 Reward -1030
Epoch 60 Reward 245
Epoch 255 Reward -1075
Epoch 376 Reward -1205
Epoch 190 Reward -2
Epoch 264 Reward -400
Epoch 124 Reward -98
Epoch 137 Reward 222
Epoch 175 Reward -185
Epoch 236 Reward -534
Epoch 243 Reward -118
Epoch 148 Reward -5
Epoch 47 Reward 366
Epoch 94 Reward 328
Epoch 33 Reward 452
Epoch 118 Reward -29
Epoch 230 Reward -222
Epoch 87 Reward 218
Epoch 84 Reward 203
Epoch 77 Reward 255
Epoch 67 Reward 265
Epoch 281 Reward -606
Epoch 184 Reward -14
Epoch 590 Reward -2337
Epoch 660 Reward -904
Epoch 276 Reward -817
Epoch 52 Reward 397
Epoch 312 Reward -682
Epoch 74 Reward 231
Epoch 28

KeyboardInterrupt: 

In [16]:
# tensorflow와 그 안의 모델을 실행
with tf.Session() as sess:
    # 저장 객체 생성
    saver = tf.train.Saver()
    
    # 저장된 모델 로드
    saver.restore(sess, tf.train.latest_checkpoint('./')) 

    done = False
    obs = env.reset()
    total_reward = 0


    #상태가 최종 상태가 아닌 동안
    while not done:

        display.clear_output(wait=True)
        display.display(plt.gcf())
        env.render()

        # 전처리된 게임 화면 가져오기
        obs = env.render('dqn')

        # 게임 화면을 피드하고 각 작업에 대한 Q 값을 가져오기
        actions = mainQ_outputs.eval(feed_dict={X:[obs], in_training_mode: False})

        # 행동 가져오기
        action = np.argmax(actions, axis =-1)

        #행동을 수행하고 다음 상태인 next_obs로 이동하여 보상을 받는다.
        next_obs, reward, done, _ = env.step(action)
        total_reward += reward
        print("Episode Reward= ", total_reward)
        time.sleep(0.5)


o - - - - - - - - - - - - - - - - - o
| P |       |           |           |
|           | - |   |   | - | - |   |
|   |     2         |   |           |
| - |                       | - |   |
|       |   |                       |
|   |                       | - |   |
|   |                       |       |
|   | - | - | - |           | - |   |
|                         0     | - |
o - - - - - - - - - - - - - - - - - o

Episode Reward=  -380


KeyboardInterrupt: 